In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [4]:
# создание сессии

spark = SparkSession.builder.appName('Practice').getOrCreate()

In [5]:
spark

In [6]:
# конвертирую excel в csv

import pandas as pd

df_excel = pd.read_excel('cvm_srv.xlsx')

df_excel.to_csv('cvm_srv.csv', index=False)

In [16]:
# чтение фрейма данных

# InferSchema - автоматическое определение типов данных при чтении файла

df_spark = spark.read.option('header', 'true').csv('cvm_srv.csv', inferSchema=True)

In [17]:
# вывод результата

df_spark.show(1)

+-----------+----------------+--------------------+-----------+----------------+-----+---------+----------+------+--------+----+
|campaign_cd|     campaign_nm|            name_cvm|global_code|     global_name|regid|  service|       cvm|contur| product|dayy|
+-----------+----------------+--------------------+-----------+----------------+-----+---------+----------+------+--------+----+
|CAMP1026252|1757_RTM_B2B_MNR|50 SMS в поездках...|   PE0910.1|100 SMS в Европе|   67|46355.067|CVMB2B-865|   b2c|Mob_core|  30|
+-----------+----------------+--------------------+-----------+----------------+-----+---------+----------+------+--------+----+
only showing top 1 row



In [18]:
# вывод схемы

df_spark.printSchema()

root
 |-- campaign_cd: string (nullable = true)
 |-- campaign_nm: string (nullable = true)
 |-- name_cvm: string (nullable = true)
 |-- global_code: string (nullable = true)
 |-- global_name: string (nullable = true)
 |-- regid: integer (nullable = true)
 |-- service: double (nullable = true)
 |-- cvm: string (nullable = true)
 |-- contur: string (nullable = true)
 |-- product: string (nullable = true)
 |-- dayy: integer (nullable = true)



In [20]:
# получение колонок

df_spark.columns

['campaign_cd',
 'campaign_nm',
 'name_cvm',
 'global_code',
 'global_name',
 'regid',
 'service',
 'cvm',
 'contur',
 'product',
 'dayy']

In [21]:
# получение кортежа из строк
df_spark.head(1)

[Row(campaign_cd='CAMP1026252', campaign_nm='1757_RTM_B2B_MNR', name_cvm='50 SMS в поездках по миру', global_code='PE0910.1', global_name='100 SMS в Европе', regid=67, service=46355.067, cvm='CVMB2B-865', contur='b2c', product='Mob_core', dayy=30)]

In [24]:
# Выборка по полю

df_spark.select('regid', 'dayy').show()

+-----+----+
|regid|dayy|
+-----+----+
|   67|  30|
|   67|  30|
|   68|  30|
|   68|  30|
|   66|  30|
|   71|  30|
|   60|  30|
|   66|  30|
|   68|  30|
|   71|  30|
|   66|  30|
|   67|  30|
|   60|  30|
|   71|  30|
|   60|  30|
|   68|  30|
|   68|  30|
|   67|  30|
|   60|  30|
|   66|  30|
+-----+----+
only showing top 20 rows



In [29]:
# Добавление поля

df_spark = df_spark.withColumn("new_column", df_spark['regid']+1)

In [32]:
# Удаление поля

df_spark = df_spark.drop('new_column')

In [33]:
# Переименование поля

df_spark.withColumnRenamed("regid", "regid_new")

DataFrame[campaign_cd: string, campaign_nm: string, name_cvm: string, global_code: string, global_name: string, regid_new: int, service: double, cvm: string, contur: string, product: string, dayy: int]